---
## Setup: Import Required Libraries

In [7]:
import os
import sys

# Setup SUMO_HOME
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
    print(f"✓ SUMO_HOME: {os.environ['SUMO_HOME']}")
else:
    print("⚠ SUMO_HOME not set - trying to continue anyway...")
    # Try common locations
    for path in ['/usr/share/sumo', '/usr/local/share/sumo', 'C:/Program Files/SUMO']:
        if os.path.exists(path):
            os.environ['SUMO_HOME'] = path
            sys.path.append(os.path.join(path, 'tools'))
            print(f"✓ Found SUMO at: {path}")
            break

try:
    import sumolib
    import traci
    import traci.constants as tc
    print("✓ sumolib and TraCI imported successfully")
except ImportError as e:
    print(f"✗ Import error: {e}")

✓ SUMO_HOME: /usr/share/sumo
✓ sumolib and TraCI imported successfully


---
## Part 1: Creating a Simple Network with Pandas

Let's create a simple network from scratch using pandas DataFrames!

In [8]:
# Create network data using pandas
import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom

def prettify_xml(elem):
    """Return a pretty-printed XML string"""
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

# Create nodes DataFrame
nodes_df = pd.DataFrame({
    'id': ['n0', 'n1', 'n2', 'n3'],
    'x': [0, 200, 400, 200],
    'y': [0, 0, 0, 200]
})

print("📊 Nodes DataFrame:")
print(nodes_df)
print()

# Convert DataFrame to XML
nodes_xml = ET.Element('nodes')
for _, row in nodes_df.iterrows():
    ET.SubElement(nodes_xml, 'node', 
                  id=row['id'], 
                  x=str(row['x']), 
                  y=str(row['y']))

# Save nodes file
with open('demo.nod.xml', 'w') as f:
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write(prettify_xml(nodes_xml))

print("✓ Created demo.nod.xml with nodes:")
for _, row in nodes_df.iterrows():
    print(f"  {row['id']}: ({row['x']}, {row['y']})")

📊 Nodes DataFrame:
   id    x    y
0  n0    0    0
1  n1  200    0
2  n2  400    0
3  n3  200  200

✓ Created demo.nod.xml with nodes:
  n0: (0, 0)
  n1: (200, 0)
  n2: (400, 0)
  n3: (200, 200)


In [3]:
# Create edges DataFrame
edges_df = pd.DataFrame({
    'id': ['e0', 'e1', 'e2'],
    'from': ['n0', 'n1', 'n1'],
    'to': ['n1', 'n2', 'n3'],
    'numLanes': [2, 2, 1],
    'speed_ms': [13.89, 13.89, 11.11],  # m/s
    'speed_kmh': [50, 50, 40],  # km/h (for reference)
    'priority': [1, 1, 0]
})

print("📊 Edges DataFrame:")
print(edges_df)
print()

# Convert DataFrame to XML
edges_xml = ET.Element('edges')
for _, row in edges_df.iterrows():
    ET.SubElement(edges_xml, 'edge',
                  id=row['id'],
                  **{'from': row['from'], 'to': row['to']},
                  numLanes=str(row['numLanes']),
                  speed=str(row['speed_ms']),
                  priority=str(row['priority']))

# Save edges file
with open('demo.edg.xml', 'w') as f:
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write(prettify_xml(edges_xml))

print("✓ Created demo.edg.xml with edges:")
for _, row in edges_df.iterrows():
    print(f"  {row['id']}: {row['from']}→{row['to']}, {row['numLanes']} lanes, {row['speed_kmh']} km/h")

📊 Edges DataFrame:
   id from  to  numLanes  speed_ms  speed_kmh  priority
0  e0   n0  n1         2     13.89         50         1
1  e1   n1  n2         2     13.89         50         1
2  e2   n1  n3         1     11.11         40         0

✓ Created demo.edg.xml with edges:
  e0: n0→n1, 2 lanes, 50 km/h
  e1: n1→n2, 2 lanes, 50 km/h
  e2: n1→n3, 1 lanes, 40 km/h


In [6]:
# Build the network using netconvert
import subprocess

try:
    result = subprocess.run([
        'netconvert',
        '--node-files=/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/demo.nod.xml',
        '--edge-files=/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/demo.edg.xml',
        '--output-file=demo.net.xml',
        '--no-warnings'
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✓ Network created successfully: demo.net.xml")
    else:
        print(f"✗ Error creating network: {result.stderr}")
except FileNotFoundError:
    print("✗ netconvert not found. Make sure SUMO is installed and in PATH.")

✗ Error creating network: Error: XML or TEXT declaration must start at line 1, column 1
 In file '/mnt/Data1/Python_Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/demo.nod.xml'
 At line/column 3/7.

Error: No nodes loaded.
Quitting (on error).



---
## Part 2: Analyzing the Network with sumolib

Now let's load and analyze our network!

In [9]:
# Load the network
try:
    net = sumolib.net.readNet('demo.net.xml')
    print("✓ Network loaded successfully!\n")
    
    # Basic statistics
    edges = net.getEdges()
    nodes = net.getNodes()
    
    print(f"📊 Network Statistics:")
    print(f"   Total edges: {len(edges)}")
    print(f"   Total nodes: {len(nodes)}")
    print(f"   Total lanes: {sum(len(e.getLanes()) for e in edges)}")
    
except Exception as e:
    print(f"✗ Error loading network: {e}")

✗ Error loading network: unknown url type: 'demo.net.xml'


In [ ]:
# Analyze each edge in detail
print("\n🛣️  Edge Details:")
print("-" * 70)

for edge in net.getEdges():
    if not edge.isSpecial():  # Skip internal edges
        print(f"\nEdge: {edge.getID()}")
        print(f"  Length: {edge.getLength():.2f} m")
        print(f"  Speed limit: {edge.getSpeed():.2f} m/s ({edge.getSpeed() * 3.6:.1f} km/h)")
        print(f"  Number of lanes: {len(edge.getLanes())}")
        print(f"  From: {edge.getFromNode().getID()} → To: {edge.getToNode().getID()}")
        print(f"  Priority: {edge.getPriority()}")
        
        # Lane details
        for i, lane in enumerate(edge.getLanes()):
            print(f"    Lane {i}: {lane.getID()}, width={lane.getWidth():.2f}m")

In [ ]:
# Calculate network bounds and geometry
print("\n📐 Network Geometry:")
print("-" * 70)

bounds = net.getBBoxXY()
print(f"Bounding box:")
print(f"  Min: ({bounds[0][0]:.2f}, {bounds[0][1]:.2f})")
print(f"  Max: ({bounds[1][0]:.2f}, {bounds[1][1]:.2f})")
print(f"  Width: {bounds[1][0] - bounds[0][0]:.2f} m")
print(f"  Height: {bounds[1][1] - bounds[0][1]:.2f} m")

# Total network length
total_length = sum(edge.getLength() for edge in edges if not edge.isSpecial())
print(f"\nTotal road length: {total_length:.2f} m ({total_length/1000:.2f} km)")

In [ ]:
# Find neighboring edges to a point
print("\n🎯 Spatial Queries:")
print("-" * 70)

# Pick a point in the middle
test_point = (200, 100)
radius = 150

print(f"Finding edges within {radius}m of point {test_point}:")
nearby_edges = net.getNeighboringEdges(test_point[0], test_point[1], radius)

for edge, distance in nearby_edges:
    if not edge.isSpecial():
        print(f"  {edge.getID()}: {distance:.2f}m away")

---
## Part 3: Create Routes and Vehicles with Pandas

Let's create some traffic using pandas DataFrames!

In [ ]:
# Define vehicle types using pandas
vehicle_types_df = pd.DataFrame({
    'id': ['car', 'truck'],
    'accel': [2.6, 1.3],
    'decel': [4.5, 4.0],
    'sigma': [0.5, 0.5],
    'length': [5, 12],
    'maxSpeed': [50, 30],
    'color': ['1,0,0', '0,0,1']
})

print("📊 Vehicle Types:")
print(vehicle_types_df)
print()

# Define routes using pandas
routes_df = pd.DataFrame({
    'id': ['route0', 'route1'],
    'edges': ['e0 e1', 'e0']
})

print("📊 Routes:")
print(routes_df)
print()

# Define vehicles using pandas
vehicles_df = pd.DataFrame({
    'id': ['car_0', 'car_1', 'car_2', 'truck_0', 'car_3', 'car_4'],
    'type': ['car', 'car', 'car', 'truck', 'car', 'car'],
    'route': ['route0', 'route0', 'route0', 'route0', 'route0', 'route1'],
    'depart': [0, 5, 10, 15, 20, 25],
    'color': ['1,0,0', '1,0.5,0', '1,1,0', '0,0,1', '0,1,0', '1,0,1']
})

print("📊 Vehicles:")
print(vehicles_df)
print()

# Convert to XML
routes_root = ET.Element('routes')

# Add vehicle types
for _, vtype in vehicle_types_df.iterrows():
    ET.SubElement(routes_root, 'vType',
                  id=vtype['id'],
                  accel=str(vtype['accel']),
                  decel=str(vtype['decel']),
                  sigma=str(vtype['sigma']),
                  length=str(vtype['length']),
                  maxSpeed=str(vtype['maxSpeed']),
                  color=vtype['color'])

# Add routes
for _, route in routes_df.iterrows():
    ET.SubElement(routes_root, 'route',
                  id=route['id'],
                  edges=route['edges'])

# Add vehicles
for _, vehicle in vehicles_df.iterrows():
    ET.SubElement(routes_root, 'vehicle',
                  id=vehicle['id'],
                  type=vehicle['type'],
                  route=vehicle['route'],
                  depart=str(vehicle['depart']),
                  color=vehicle['color'])

# Save to file
with open('demo.rou.xml', 'w') as f:
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write(prettify_xml(routes_root))

print("✓ Created demo.rou.xml with:")
print(f"  - {len(vehicle_types_df)} vehicle types")
print(f"  - {len(routes_df)} routes")
print(f"  - {len(vehicles_df)} vehicles")

In [ ]:
# Create SUMO configuration file
config_xml = """<?xml version="1.0" encoding="UTF-8"?>
<configuration>
    <input>
        <net-file value="demo.net.xml"/>
        <route-files value="demo.rou.xml"/>
    </input>
    <time>
        <begin value="0"/>
        <end value="100"/>
        <step-length value="0.1"/>
    </time>
    <processing>
        <time-to-teleport value="-1"/>
    </processing>
</configuration>
"""

with open('demo.sumocfg', 'w') as f:
    f.write(config_xml)

print("✓ Created demo.sumocfg")
print("  Simulation: 0-100 seconds, step=0.1s")

---
## Part 4: Run Simulation with TraCI - Basic Monitoring

Let's run the simulation and monitor what happens!

In [ ]:
# Run simulation and collect data in pandas DataFrame
import time as pytime

# Use 'sumo' for headless, 'sumo-gui' to see visualization
sumo_binary = 'sumo'

# Start TraCI
traci.start([sumo_binary, '-c', 'demo.sumocfg', '--no-warnings', '--no-step-log'])

print("🚗 Starting simulation...\n")

# Storage for data - will convert to DataFrame later
vehicle_records = []
departed_vehicles = set()

try:
    step = 0
    while traci.simulation.getMinExpectedNumber() > 0 and step < 1000:
        traci.simulationStep()
        current_time = traci.simulation.getTime()
        
        # Get all vehicles currently in simulation
        vehicles = traci.vehicle.getIDList()
        
        # Track new departures
        for veh_id in vehicles:
            if veh_id not in departed_vehicles:
                departed_vehicles.add(veh_id)
                veh_type = traci.vehicle.getTypeID(veh_id)
                route = traci.vehicle.getRoute(veh_id)
                print(f"⏰ t={current_time:.1f}s: {veh_id} ({veh_type}) departed on route {route}")
        
        # Collect data every 10 steps
        if step % 10 == 0 and len(vehicles) > 0:
            for veh_id in vehicles:
                position = traci.vehicle.getPosition(veh_id)
                vehicle_records.append({
                    'time': current_time,
                    'vehicle_id': veh_id,
                    'speed': traci.vehicle.getSpeed(veh_id),
                    'x': position[0],
                    'y': position[1],
                    'lane': traci.vehicle.getLaneID(veh_id)
                })
        
        # Print status every 20 steps
        if step % 20 == 0 and len(vehicles) > 0:
            print(f"\n📍 t={current_time:.1f}s: {len(vehicles)} vehicles active")
            for veh_id in vehicles:
                speed = traci.vehicle.getSpeed(veh_id)
                lane = traci.vehicle.getLaneID(veh_id)
                pos = traci.vehicle.getLanePosition(veh_id)
                print(f"   {veh_id}: {speed*3.6:.1f} km/h on {lane} @ {pos:.1f}m")
        
        step += 1
    
    print(f"\n✓ Simulation completed at t={traci.simulation.getTime():.1f}s")
    
finally:
    traci.close()

# Convert to DataFrame
vehicle_data_df = pd.DataFrame(vehicle_records)
print(f"\n📊 Collected {len(vehicle_data_df)} data points")
print("\nFirst few records:")
print(vehicle_data_df.head(10))

In [ ]:
# Analyze the collected data using pandas
print("📊 Vehicle Journey Analysis:")
print("=" * 70)

# Convert speed to km/h
vehicle_data_df['speed_kmh'] = vehicle_data_df['speed'] * 3.6

# Group by vehicle and calculate statistics
vehicle_stats = vehicle_data_df.groupby('vehicle_id').agg({
    'time': ['min', 'max', 'count'],
    'speed_kmh': ['mean', 'max', 'min'],
    'lane': 'nunique'
}).round(2)

# Flatten column names
vehicle_stats.columns = ['_'.join(col).strip() for col in vehicle_stats.columns.values]
vehicle_stats = vehicle_stats.rename(columns={
    'time_min': 'start_time',
    'time_max': 'end_time',
    'time_count': 'samples',
    'speed_kmh_mean': 'avg_speed_kmh',
    'speed_kmh_max': 'max_speed_kmh',
    'speed_kmh_min': 'min_speed_kmh',
    'lane_nunique': 'lanes_visited'
})

# Calculate duration
vehicle_stats['duration'] = vehicle_stats['end_time'] - vehicle_stats['start_time']

# Display results
print("\n", vehicle_stats)

# Summary statistics
print(f"\n📈 Overall Statistics:")
print(f"   Average journey duration: {vehicle_stats['duration'].mean():.1f}s")
print(f"   Average speed across all vehicles: {vehicle_stats['avg_speed_kmh'].mean():.1f} km/h")
print(f"   Highest speed recorded: {vehicle_stats['max_speed_kmh'].max():.1f} km/h")

---
## Part 5: Vehicle Control Example

Let's control vehicle behavior in real-time!

In [ ]:
# Run simulation with vehicle control
traci.start([sumo_binary, '-c', 'demo.sumocfg', '--no-warnings', '--no-step-log'])

print("🎮 Running simulation with vehicle control...\n")

controlled_vehicles = set()

try:
    step = 0
    while traci.simulation.getMinExpectedNumber() > 0 and step < 1000:
        traci.simulationStep()
        current_time = traci.simulation.getTime()
        
        vehicles = traci.vehicle.getIDList()
        
        for veh_id in vehicles:
            # Control specific vehicles
            if veh_id == 'car_1' and veh_id not in controlled_vehicles:
                controlled_vehicles.add(veh_id)
                print(f"🎯 t={current_time:.1f}s: Taking control of {veh_id}")
            
            # Apply different controls based on time
            if veh_id in controlled_vehicles:
                if 10 <= current_time < 15:
                    # Force slow down
                    traci.vehicle.setSpeed(veh_id, 3.0)  # 10.8 km/h
                    if step % 10 == 0:
                        print(f"   t={current_time:.1f}s: Slowing down {veh_id} to 3 m/s")
                
                elif 15 <= current_time < 20:
                    # Speed up
                    traci.vehicle.setSpeed(veh_id, 15.0)  # 54 km/h
                    if step % 10 == 0:
                        print(f"   t={current_time:.1f}s: Speeding up {veh_id} to 15 m/s")
                
                elif current_time >= 20:
                    # Return to normal
                    traci.vehicle.setSpeed(veh_id, -1)  # -1 = automatic
                    if step % 10 == 0:
                        print(f"   t={current_time:.1f}s: {veh_id} back to automatic control")
            
            # Change color based on speed
            speed = traci.vehicle.getSpeed(veh_id)
            if speed < 1.0:
                traci.vehicle.setColor(veh_id, (255, 0, 0, 255))  # Red when stopped
            elif speed < 5.0:
                traci.vehicle.setColor(veh_id, (255, 255, 0, 255))  # Yellow when slow
            else:
                traci.vehicle.setColor(veh_id, (0, 255, 0, 255))  # Green when moving
        
        step += 1
    
    print(f"\n✓ Control simulation completed")
    
finally:
    traci.close()

---
## Part 6: Using Subscriptions for Efficiency

Demonstrate the performance difference between getters and subscriptions.

In [ ]:
# Method 1: Using individual getters (slower)
import time

traci.start([sumo_binary, '-c', 'demo.sumocfg', '--no-warnings', '--no-step-log'])

print("⏱️  Method 1: Using individual getters")

start_time = time.time()
step_count = 0

try:
    while traci.simulation.getMinExpectedNumber() > 0 and step_count < 500:
        traci.simulationStep()
        
        for veh_id in traci.vehicle.getIDList():
            # Multiple individual getter calls
            speed = traci.vehicle.getSpeed(veh_id)
            position = traci.vehicle.getPosition(veh_id)
            lane = traci.vehicle.getLaneID(veh_id)
            road = traci.vehicle.getRoadID(veh_id)
        
        step_count += 1
    
    getter_time = time.time() - start_time
    print(f"✓ Completed {step_count} steps in {getter_time:.3f}s")
    
finally:
    traci.close()

In [ ]:
# Method 2: Using subscriptions (faster)
traci.start([sumo_binary, '-c', 'demo.sumocfg', '--no-warnings', '--no-step-log'])

print("\n⏱️  Method 2: Using subscriptions")

subscribed_vehicles = set()
start_time = time.time()
step_count = 0

try:
    while traci.simulation.getMinExpectedNumber() > 0 and step_count < 500:
        traci.simulationStep()
        
        # Subscribe to new vehicles
        for veh_id in traci.vehicle.getIDList():
            if veh_id not in subscribed_vehicles:
                traci.vehicle.subscribe(veh_id, [
                    tc.VAR_SPEED,
                    tc.VAR_POSITION,
                    tc.VAR_LANE_ID,
                    tc.VAR_ROAD_ID
                ])
                subscribed_vehicles.add(veh_id)
        
        # Get all data at once
        for veh_id in traci.vehicle.getIDList():
            results = traci.vehicle.getSubscriptionResults(veh_id)
            speed = results[tc.VAR_SPEED]
            position = results[tc.VAR_POSITION]
            lane = results[tc.VAR_LANE_ID]
            road = results[tc.VAR_ROAD_ID]
        
        step_count += 1
    
    subscription_time = time.time() - start_time
    print(f"✓ Completed {step_count} steps in {subscription_time:.3f}s")
    
    print(f"\n📈 Performance comparison:")
    print(f"   Getters: {getter_time:.3f}s")
    print(f"   Subscriptions: {subscription_time:.3f}s")
    print(f"   Speedup: {getter_time/subscription_time:.2f}x faster!")
    
finally:
    traci.close()

---
## Part 7: Adding Vehicles Dynamically

Create vehicles during simulation runtime!

In [ ]:
# Add vehicles dynamically during simulation - track in DataFrame
traci.start([sumo_binary, '-c', 'demo.sumocfg', '--no-warnings', '--no-step-log'])

print("🚗 Adding vehicles dynamically...\n")

# Define a route first
traci.route.add("dynamic_route", ["e0", "e1"])

# DataFrame to track added vehicles
added_vehicles_df = pd.DataFrame(columns=['vehicle_id', 'added_at', 'color_r', 'color_g', 'color_b'])

# Color options
colors = [
    {'r': 255, 'g': 0, 'b': 0},    # Red
    {'r': 0, 'g': 255, 'b': 0},    # Green
    {'r': 0, 'g': 0, 'b': 255},    # Blue
    {'r': 255, 'g': 255, 'b': 0}   # Yellow
]

try:
    step = 0
    while step < 600:
        traci.simulationStep()
        current_time = traci.simulation.getTime()
        
        # Add a new vehicle every 5 seconds
        if step % 50 == 0 and step > 0:
            veh_id = f"dynamic_{len(added_vehicles_df)}"
            try:
                traci.vehicle.add(
                    veh_id,
                    "dynamic_route",
                    typeID="car",
                    depart=str(int(current_time)),
                    departLane="best",
                    departSpeed="max"
                )
                
                # Set custom color
                color = colors[len(added_vehicles_df) % len(colors)]
                traci.vehicle.setColor(veh_id, (color['r'], color['g'], color['b'], 255))
                
                # Add to DataFrame
                new_vehicle = pd.DataFrame([{
                    'vehicle_id': veh_id,
                    'added_at': current_time,
                    'color_r': color['r'],
                    'color_g': color['g'],
                    'color_b': color['b']
                }])
                added_vehicles_df = pd.concat([added_vehicles_df, new_vehicle], ignore_index=True)
                
                print(f"➕ t={current_time:.1f}s: Added {veh_id} (total: {len(added_vehicles_df)})")
                
            except traci.exceptions.TraCIException as e:
                print(f"   ⚠️  Could not add vehicle: {e}")
        
        # Print status
        if step % 100 == 0:
            active = len(traci.vehicle.getIDList())
            print(f"📊 t={current_time:.1f}s: {active} vehicles active, {len(added_vehicles_df)} added dynamically")
        
        step += 1
    
    print(f"\n✓ Successfully added {len(added_vehicles_df)} vehicles dynamically")
    print("\n📊 Added Vehicles Summary:")
    print(added_vehicles_df)
    
finally:
    traci.close()

---
## Part 8: Analyzing Traffic Flow

Calculate traffic flow metrics!

In [ ]:
# Analyze traffic flow on edges - store in DataFrame
traci.start([sumo_binary, '-c', 'demo.sumocfg', '--no-warnings', '--no-step-log'])

print("📊 Analyzing traffic flow...\n")

# Get non-special edges
edge_ids = [edge.getID() for edge in net.getEdges() if not edge.isSpecial()]

# Storage for samples
flow_samples = []

try:
    step = 0
    while traci.simulation.getMinExpectedNumber() > 0 and step < 1000:
        traci.simulationStep()
        current_time = traci.simulation.getTime()
        
        # Sample data every 10 steps
        if step % 10 == 0:
            for edge_id in edge_ids:
                vehicles_on_edge = traci.edge.getLastStepVehicleIDs(edge_id)
                
                if vehicles_on_edge:
                    total_speed = sum(traci.vehicle.getSpeed(v) for v in vehicles_on_edge)
                    avg_speed = total_speed / len(vehicles_on_edge)
                else:
                    avg_speed = 0
                
                flow_samples.append({
                    'time': current_time,
                    'edge_id': edge_id,
                    'vehicle_count': len(vehicles_on_edge),
                    'avg_speed_ms': avg_speed,
                    'occupied': 1 if vehicles_on_edge else 0
                })
        
        step += 1
    
    print(f"✓ Simulation completed")
    
    # Convert to DataFrame
    flow_df = pd.DataFrame(flow_samples)
    flow_df['avg_speed_kmh'] = flow_df['avg_speed_ms'] * 3.6
    
    # Calculate statistics per edge
    print("\n📈 Traffic Flow Analysis Results:")
    print("=" * 70)
    
    edge_stats = flow_df.groupby('edge_id').agg({
        'vehicle_count': ['sum', 'mean'],
        'avg_speed_kmh': 'mean',
        'occupied': 'mean'
    }).round(2)
    
    edge_stats.columns = ['total_vehicles', 'avg_vehicles', 'avg_speed_kmh', 'occupancy']
    edge_stats['occupancy_pct'] = (edge_stats['occupancy'] * 100).round(1)
    
    # Add edge properties
    for edge_id in edge_stats.index:
        edge = net.getEdge(edge_id)
        edge_stats.loc[edge_id, 'num_lanes'] = len(edge.getLanes())
        edge_stats.loc[edge_id, 'speed_limit_kmh'] = edge.getSpeed() * 3.6
    
    print("\n", edge_stats)
    
    # Show sample data
    print("\n📊 Sample Flow Data (first 10 records):")
    print(flow_df.head(10))
    
finally:
    traci.close()

---
## Part 9: Complete Example - Simple Adaptive Control

Implement a basic adaptive speed control system!

In [ ]:
# Adaptive speed control - track in DataFrame
traci.start([sumo_binary, '-c', 'demo.sumocfg', '--no-warnings', '--no-step-log'])

print("🤖 Running adaptive speed control simulation...\n")

class SimpleAdaptiveControl:
    def __init__(self, edge_id, max_vehicles=4):
        self.edge_id = edge_id
        self.max_vehicles = max_vehicles
        self.speed_limit_default = net.getEdge(edge_id).getSpeed()
        self.control_log = []
    
    def update(self, current_time):
        # Count vehicles on edge
        vehicles = traci.edge.getLastStepVehicleIDs(self.edge_id)
        num_vehicles = len(vehicles)
        
        # Calculate density
        density = num_vehicles / self.max_vehicles
        
        # Adjust speed limit based on density
        if density > 0.8:
            new_speed = self.speed_limit_default * 0.6  # Reduce to 60%
            action = "REDUCE"
        elif density > 0.5:
            new_speed = self.speed_limit_default * 0.8  # Reduce to 80%
            action = "LOWER"
        else:
            new_speed = self.speed_limit_default  # Normal
            action = "NORMAL"
        
        # Apply speed limit to all vehicles on edge
        for veh_id in vehicles:
            max_speed = traci.vehicle.getMaxSpeed(veh_id)
            traci.vehicle.setMaxSpeed(veh_id, min(new_speed, max_speed))
        
        self.control_log.append({
            'time': current_time,
            'vehicles': num_vehicles,
            'density': density,
            'speed_limit_ms': new_speed,
            'speed_limit_kmh': new_speed * 3.6,
            'action': action
        })
        
        return num_vehicles, action, new_speed
    
    def get_log_df(self):
        """Return control log as DataFrame"""
        return pd.DataFrame(self.control_log)

# Create controller for edge e0
controller = SimpleAdaptiveControl('e0')

try:
    step = 0
    while traci.simulation.getMinExpectedNumber() > 0 and step < 1000:
        traci.simulationStep()
        current_time = traci.simulation.getTime()
        
        # Update controller every 5 steps
        if step % 5 == 0:
            num_veh, action, speed = controller.update(current_time)
            
            if step % 50 == 0:
                print(f"t={current_time:5.1f}s | Vehicles: {num_veh} | Action: {action:6s} | Speed: {speed*3.6:.1f} km/h")
        
        step += 1
    
    print(f"\n✓ Adaptive control simulation completed")
    
    # Get control log as DataFrame
    control_df = controller.get_log_df()
    
    # Analyze control actions
    print("\n📊 Control Action Summary:")
    action_summary = control_df['action'].value_counts()
    action_pct = (control_df['action'].value_counts(normalize=True) * 100).round(1)
    
    summary_df = pd.DataFrame({
        'count': action_summary,
        'percentage': action_pct
    })
    print(summary_df)
    
    # Statistics by action type
    print("\n📈 Average Density by Action:")
    density_stats = control_df.groupby('action')['density'].agg(['mean', 'min', 'max']).round(3)
    print(density_stats)
    
    print("\n📊 Control Log Sample (first 10 records):")
    print(control_df.head(10))
    
finally:
    traci.close()

---
## Summary

### What We Demonstrated:

✅ **Created** network data with pandas DataFrames  
✅ **Analyzed** network structure with sumolib  
✅ **Defined** routes and vehicles using pandas  
✅ **Ran** simulations with TraCI  
✅ **Collected** vehicle behavior data in DataFrames  
✅ **Controlled** vehicle speeds and colors  
✅ **Compared** performance: getters vs subscriptions  
✅ **Tracked** dynamically added vehicles with pandas  
✅ **Analyzed** traffic flow metrics using pandas  
✅ **Implemented** adaptive speed control with DataFrame logging  

### Key Benefits of Using Pandas:

- 📊 **Structured data** - Easy to view and manipulate
- 🔍 **Powerful analysis** - Group by, aggregate, filter
- 📈 **Quick statistics** - Mean, max, min, count
- 💾 **Easy export** - CSV, Excel, JSON formats
- 🎨 **Visualization ready** - Works with matplotlib, seaborn, plotly

### Try These Next:

1. Export DataFrames to CSV for further analysis
2. Create visualizations with matplotlib/plotly
3. Analyze time-series patterns in vehicle speeds
4. Compare different control strategies using pandas
5. Build more complex networks from CSV files
6. Use pandas to generate large-scale vehicle populations

---

**Keep experimenting with pandas and SUMO!** 🚗📊💨